In [27]:
import pandas as pd
from scipy import stats

In [28]:
all_sheets = pd.read_excel("./data/gemma-3-12b-single-sun.xlsx", sheet_name=None)
# List to hold dataframes with prefixed columns
dfs = []
for sheet_name, df in all_sheets.items():
    # Prefix columns with sheet name
    df_prefixed = df.add_prefix(f"{sheet_name}_")
    dfs.append(df_prefixed)
# Concatenate all dataframes column-wise
gemma = pd.concat(dfs, axis=1)
del dfs  # Free up memory

In [ ]:
gemma = gemma.set_index("letters_new_names")
gemma.index.rename("names")
# Extract scores from Gemma spreadsheet
gemma = gemma.loc[:, gemma.filter(regex="score").columns]

gemma.info()

<class 'pandas.core.frame.DataFrame'>
Index: 562 entries, Oliviu Barbu to Loredana Diaconu
Data columns (total 90 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   letters_formal_and_structural_score_1                   562 non-null    int64  
 1   letters_formal_and_structural_score_2                   562 non-null    int64  
 2   letters_formal_and_structural_score_3                   562 non-null    int64  
 3   letters_formal_and_structural_score_4                   562 non-null    int64  
 4   letters_formal_and_structural_score_5                   562 non-null    int64  
 5   letters_formal_and_structural_final_score               562 non-null    object 
 6   letters_relevance_and_thematic_fit_score_1              562 non-null    int64  
 7   letters_relevance_and_thematic_fit_score_2              562 non-null    int64  
 8   letters_relevance_and

In [ ]:
gemma_scores = pd.DataFrame()
# Recalculate mean scores
for score in gemma.filter(regex="final_score").columns:
    label = score.replace("_final_score", "")
    cols = gemma.filter(regex=f"{label}_score").columns
    gemma_scores[label] = gemma[cols].mean(axis=1)

gemma_scores.fillna(0, inplace=True)
gemma_scores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 562 entries, Oliviu Barbu to Loredana Diaconu
Data columns (total 15 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   letters_formal_and_structural               562 non-null    float64
 1   letters_relevance_and_thematic_fit          562 non-null    float64
 2   letters_professionalism_and_research        562 non-null    float64
 3   letters_uniqueness_and_individuality        562 non-null    float64
 4   letters_potential_and_developmental         562 non-null    float64
 5   presentations_formal_and_structural         562 non-null    float64
 6   presentations_relevance_and_thematic_fit    562 non-null    float64
 7   presentations_professionalism_and_research  562 non-null    float64
 8   presentations_uniqueness_and_individuality  562 non-null    float64
 9   presentations_potential_and_developmental   562 non-null    float64


In [ ]:
human_scores = pd.read_parquet("./data/dataset.parquet")
# Filter the columns to keep only the relevant scores
human_scores = human_scores[["prof_score", "all_phd_1", "all_phd_2", "offline_test_total"]]
human_scores["phd_score"] = human_scores[["all_phd_1", "all_phd_2"]].mean(axis=1)
human_scores = human_scores.drop(columns=["all_phd_1", "all_phd_2"])

human_scores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 577 entries, Adam Mazilescu to Ștefania Albu
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   prof_score          98 non-null     float64
 1   offline_test_total  48 non-null     float64
 2   phd_score           567 non-null    float64
dtypes: float64(3)
memory usage: 18.0+ KB


In [32]:
SELECTED_COLS = ["phd_score", "prof_score", "offline_test_total"]
# SELECTED_COLS = ["phd_score", "prof_score"]
# SELECTED_COLS = ["offline_test_total"]

In [33]:
pearson = pd.DataFrame(index=gemma_scores.columns, columns=SELECTED_COLS)
temp = pd.merge(human_scores, gemma_scores, left_index=True, right_index=True, how="inner")
for gemma_col in pearson.index:
    for human_col in pearson.columns:
        # Calculate Pearson correlation coefficient and p-value
        selection = temp[[gemma_col, human_col]].dropna(axis=0)
        coef, pvalue = stats.pearsonr(selection[gemma_col], selection[human_col])
        pearson.loc[gemma_col, human_col] = f"{coef:.4f} ({pvalue:.4f})"

In [ ]:
spearman = pd.DataFrame(index=gemma_scores.columns, columns=SELECTED_COLS)
temp = pd.merge(gemma_scores, human_scores, left_index=True, right_index=True, how="inner")
for gemma_col in spearman.index:
    for human_col in spearman.columns:
        # Calculate Spearman correlation coefficient and p-value
        selection = temp[[gemma_col, human_col]].dropna(axis=0)
        coef, pvalue = stats.spearmanr(selection[gemma_col], selection[human_col])
        spearman.loc[gemma_col, human_col] = f"{coef:.4f} ({pvalue:.4f})"

In [35]:
pd.concat([spearman, pearson], axis=1, keys=["spearman", "pearson"])

spearman  \
                                                   phd_score   
letters_formal_and_structural                0.1382 (0.0010)   
letters_relevance_and_thematic_fit           0.2928 (0.0000)   
letters_professionalism_and_research         0.2533 (0.0000)   
letters_uniqueness_and_individuality         0.3026 (0.0000)   
letters_potential_and_developmental          0.3064 (0.0000)   
presentations_formal_and_structural          0.0420 (0.3188)   
presentations_relevance_and_thematic_fit     0.0375 (0.3734)   
presentations_professionalism_and_research   0.0338 (0.4223)   
presentations_uniqueness_and_individuality   0.0710 (0.0918)   
presentations_potential_and_developmental    0.0497 (0.2382)   
cv_formal_and_structural                     0.0350 (0.4064)   
cv_relevance_and_thematic_fit                0.0423 (0.3157)   
cv_professionalism_and_research              0.0494 (0.2406)   
cv_uniqueness_and_individuality             -0.0108 (0.7969)   
cv_potential_and_developmental               0.0307 (0.4668)   

                                                              \
                                                  prof_score   
letters_formal_and_structural               -0.1262 (0.2156)   
letters_relevance_and_thematic_fit           0.1350 (0.1850)   
letters_professionalism_and_research         0.0033 (0.9740)   
letters_uniqueness_and_individuality         0.0279 (0.7848)   
letters_potential_and_developmental         -0.0041 (0.9680)   
presentations_formal_and_structural          0.0213 (0.8349)   
presentations_relevance_and_thematic_fit     0.0929 (0.3627)   
presentations_professionalism_and_research   0.0739 (0.4693)   
presentations_uniqueness_and_individuality   0.1564 (0.1240)   
presentations_potential_and_developmental    0.0869 (0.3948)   
cv_formal_and_structural                     0.1454 (0.1531)   
cv_relevance_and_thematic_fit               -0.0077 (0.9398)   
cv_professionalism_and_research             -0.0338 (0.7411)   
cv_uniqueness_and_individuality              0.0091 (0.9292)   
cv_potential_and_developmental              -0.0428 (0.6754)   

                                                               \
                                           offline_test_total   
letters_formal_and_structural                -0.0797 (0.5900)   
letters_relevance_and_thematic_fit           -0.1242 (0.4003)   
letters_professionalism_and_research         -0.1359 (0.3569)   
letters_uniqueness_and_individuality         -0.1591 (0.2799)   
letters_potential_and_developmental          -0.2829 (0.0514)   
presentations_formal_and_structural          -0.1088 (0.4617)   
presentations_relevance_and_thematic_fit     -0.2617 (0.0724)   
presentations_professionalism_and_research   -0.0812 (0.5831)   
presentations_uniqueness_and_individuality   -0.1693 (0.2500)   
presentations_potential_and_developmental    -0.1932 (0.1883)   
cv_formal_and_structural                      0.0210 (0.8874)   
cv_relevance_and_thematic_fit                 0.3007 (0.0379)   
cv_professionalism_and_research               0.1340 (0.3640)   
cv_uniqueness_and_individuality               0.0607 (0.6821)   
cv_potential_and_developmental                0.1916 (0.1921)   

                                                    pearson                    \
                                                  phd_score        prof_score   
letters_formal_and_structural               0.2312 (0.0000)  -0.1063 (0.2975)   
letters_relevance_and_thematic_fit          0.3798 (0.0000)   0.0996 (0.3290)   
letters_professionalism_and_research        0.3171 (0.0000)   0.0425 (0.6774)   
letters_uniqueness_and_individuality        0.3707 (0.0000)   0.0461 (0.6523)   
letters_potential_and_developmental         0.3765 (0.0000)   0.0175 (0.8645)   
presentations_formal_and_structural         0.0349 (0.4082)   0.0697 (0.4954)   
presentations_relevance_and_thematic_fit    0.0305 (0.4699)   0.0803 (0.4316)   
presentations_professionalism_and_resea